### Instrument connection

===============Instruments in need=================

* HIOKI LR8450

三支碳棒什麼都不要接，程式執行自動校正

In [1]:
# 引入需要的包裝

import pyvisa
import time

### HIOKI8450

===============Function List=================

* 

In [2]:
class HIOKI_8450():
    def __init__(self, resource_address, visa_dll=None):
        """
        初始化儀器控制器。
        :param resource_address: 儀器的資源地址。
        :param visa_dll: 使用的 VISA DLL 路徑。如果為 None，則使用系統預設空白。
        """
        self.resource_address = resource_address
        self.visa_dll = visa_dll if visa_dll is not None else ''
        self.rm = pyvisa.ResourceManager(self.visa_dll)
        self.instrument = None
        self.connect_to_instrument()
        
    def connect_to_instrument(self):
        # 建立與儀器的連接
        self.instrument = self.rm.open_resource(self.resource_address)
        print("Connected to HIOKI8450.")
        print(self.query_identity())
        
    def query_identity(self):
         # 查詢並返回儀器的身份識別信息
        if self.instrument:
             return self.instrument.query("*IDN?")
        else:
             raise Exception("Instrument not connected. Please connect first.")

    def close_instrument(self):
        if self.instrument:
            self.instrument.close()
            print("HIOKI8450 connection closed.")
        else:
            print("No instrument to close.")

    def set_channel(self, channel, state):
        if self.instrument:
            #選擇模組中的通道是否顯示測量
            self.instrument.write(f":UNIT:STORe {channel},{state}")
            print(f"set {channel} {state}")
        else:
            print("No instrument to close.")

    def set_measurement_mode(self, channel, mode):
        if self.instrument:
            #設定測量通道的模式 [電壓、熱電偶、濕度]
            self.instrument.write(f":UNIT:INMOde {channel},{mode}")
            print(f"set mode of {channel} be {mode}")
        else:
            print("No instrument to close.")

    def set_rtd_mode(self, channel, sensor):
        if self.instrument:
            #設定熱電偶線的類型[K、J、E、T、N、R、S、C]
            self.instrument.write(f":UNIT:SENSor {channel},{sensor}")
            print(f"set the sensor kind for the {channel} be {sensor}")
        else:
            print("No instrument to close.")

    def query_data(self):
        if self.instrument:
            #直接顯示unit1此模組當下測量的所有數據
            self.instrument.write(f":MEMory:TVREAL? UNIT1")
            time.sleep(0.05)
            return self.instrument.read()
        else:
            raise Exception("Instrument not connected. Please connect first.")

    def test_cmd(self):
        if self.instrument:
            #測試通道當下的狀態
            self.instrument.write(f":MEMory:CHSTore? CH1_3")
        else:
            raise Exception("Instrument not connected. Please connect first.")

In [3]:
def main():
    
    # =========================Parmeters===============================
    """

    """
    #設置待測量之通道
    displays = [
        {'channel': 'CH1_1', 'state': 'ON', 'mode': 'TC', 'sensor': 'K'},
        {'channel': 'CH1_2', 'state': 'ON', 'mode': 'TC', 'sensor': 'K'},
        {'channel': 'CH1_3', 'state': 'OFF', 'mode': 'V', 'sensor': 'NONE'}
    ]
    
    #====================Instrument Connect Test===========================
    #從NI MAX抓儀器的VISA Resource Name
    HIOKI_instrument_address = 'ASRL3::INSTR' 
    hio = HIOKI_8450(HIOKI_instrument_address)
    #若有其他儀器串接下去...

    #============================Auto Test================================= 
    # TO DO: 這邊給你寫For迴圈自動測量的區域

    seq = [1,2,3,4,5]

    for display in displays:
        channel = display['channel']
        state = display['state']
        hio.set_channel(channel, state)
        mode = display['mode']
        hio.set_measurement_mode(channel, mode)
        sensor = display['sensor']
        hio.set_rtd_mode(channel, sensor)
    
    time.sleep(2)
    for i in seq:
        test1 = hio.query_data()
        time.sleep(1)
        print(f"result of test1 is {test1}")
        time.sleep(1)
        
    # ============================ Close Instrument=================================
    #關閉儀器
    hio.close_instrument()
    
if __name__ == "__main__":
    main()

Connected to HIOKI8450.
HIOKI,LR8450,220139566,V2.21

set CH1_1 ON
set mode of CH1_1 be TC
set the sensor kind for the CH1_1 be K
set CH1_2 ON
set mode of CH1_2 be TC
set the sensor kind for the CH1_2 be K
set CH1_3 OFF
set mode of CH1_3 be V
set the sensor kind for the CH1_3 be NONE
result of test1 is 9.99999E+99,9.99999E+99

result of test1 is 9.99999E+99,9.99999E+99

result of test1 is 9.99999E+99,9.99999E+99

result of test1 is 9.99999E+99,9.99999E+99

result of test1 is +24.2500E+00,+327.670E+00

HIOKI8450 connection closed.
